<a href="https://colab.research.google.com/github/Sebastian009/03MAIR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Sebastian Casallas Estrella  <br>
Url: https://github.com/Sebastian009/03MAIR---Algoritmos-de-Optimizacion---2023<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [65]:
#%pip install PuLP
#Librerias
import pandas as pd
import numpy as np
from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value, LpStatus

# Crear un problema de optimización
problema = LpProblem("Organizar los horarios de partidos de La Liga", LpMaximize)

# 1) Definir los Conjuntos y Parametros

# 1.1) Definir la informacion de los equipos
Categorias = ["A", "B", "C"]
Equipos_categoria = {'A': 3, 'B': 11, 'C': 6}
Audiencia = {'AA': 2.00, 'AB': 1.30, 'AC': 1.00,
             'BA': 1.30, 'BB': 0.90, 'BC': 0.75,
             'CA': 1.00, 'CB': 0.75, 'CC': 0.47}

# 1.2) Definir días y horarios disponibles
Dias = ['Viernes', 'Sábado', 'Domingo', 'Lunes']
Horarios = {'Viernes': [20],
            'Sábado': [12, 16, 18, 20],
            'Domingo': [12, 16, 18, 20],
            'Lunes': [20]}


# 2) Variables de decisión

'''
Se enumeran todas las posibles soluciones y no se tienen encuentas los partidos en donde
el equipo 1 y el equipo 2 son el mismo ademas de quitar duplicidades en las combinaciones como A1 vs B5 = B5 va A1
'''
partidos = [(eq1, num_eq1, 
             eq2, num_eq2, 
             dia, horario) for eq1 in Categorias
            for num_eq1 in range(1, Equipos_categoria[eq1]+ 1)
            for eq2 in Categorias
            for num_eq2 in range(1, Equipos_categoria[eq2]+ 1)
            if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
            for dia in Dias
            for horario in Horarios[dia]]


#Indica si el partido se juega o no para cada posible combinación
x = LpVariable.dicts("Partido", partidos, cat='Binary')

# 3) Función objetivo
problema += lpSum(Audiencia[eq1 + eq2] * x[(eq1, num_eq1, 
                                            eq2, num_eq2, 
                                            dia, horario)] for eq1 in Categorias
                  for num_eq1 in range(1, Equipos_categoria[eq1]+ 1)
                  for eq2 in Categorias
                  for num_eq2 in range(1, Equipos_categoria[eq2]+ 1)
                  if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                  for dia in Dias
                  for horario in Horarios[dia])


# 4) Restricciones

# 4.1) Cada equipo debe jugar al menos una unica vez
'''
for eq1 in Categorias:
    for num_eq1 in range(1, Equipos_categoria[eq1] + 1):
        problema += lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                          for eq2 in Categorias
                          for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                          if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                          for dia in Dias
                          for horario in Horarios[dia]) == 1
                          
'''
        
for eq1 in Categorias:
    for num_eq1 in range(1, Equipos_categoria[eq1] + 1):
        problema += (lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                          for eq2 in Categorias
                          for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                          if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                          for dia in Dias
                          for horario in Horarios[dia]) + lpSum(x[(eq2, num_eq2, eq1, num_eq1, dia, horario)]
                          for eq2 in Categorias
                          for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                          if (Categorias.index(eq2) <= Categorias.index(eq1)) and ((eq2 != eq1) or (num_eq2 != num_eq1)) and ((eq2 < eq1) or (eq2 == eq1 and num_eq2 < num_eq1))
                          for dia in Dias
                          for horario in Horarios[dia])) == 1

# 4.2) Restricciones de obligatoriedad del viernes y lunes
for dia in ['Viernes', 'Lunes']:
    problema += lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                      for eq1 in Categorias
                      for num_eq1 in range(1, Equipos_categoria[eq1] + 1)
                      for eq2 in Categorias
                      for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                      if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                      for horario in Horarios[dia]) >= 1

# Resolver el problema
status = problema.solve()
print(LpStatus[status])


# Mostrar resultados
for v in problema.variables():
    if value(v) > 0:
        print(f"{v.name}: {value(v)}")

c:\Users\Asus\anaconda3\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Optimal
Partido_('A',_1,_'A',_2,_'Sábado',_16): 1.0
Partido_('A',_3,_'B',_3,_'Lunes',_20): 1.0
Partido_('B',_1,_'B',_10,_'Sábado',_20): 1.0
Partido_('B',_11,_'C',_6,_'Domingo',_12): 1.0
Partido_('B',_2,_'C',_2,_'Domingo',_12): 1.0
Partido_('B',_4,_'C',_5,_'Sábado',_12): 1.0
Partido_('B',_5,_'C',_4,_'Viernes',_20): 1.0
Partido_('B',_6,_'C',_3,_'Sábado',_16): 1.0
Partido_('B',_7,_'B',_8,_'Viernes',_20): 1.0
Partido_('B',_9,_'C',_1,_'Domingo',_18): 1.0


'Infeasible'

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta